
#### O que são Embeddings?

**Embeddings** são representações vetoriais de textos (ou outros tipos de dados, como imagens, mas aqui focaremos em textos). Quando transformamos um texto em uma lista de números (vetor), podemos realizar diversos cálculos matemáticos para determinar similaridades e relações semânticas entre diferentes pedaços de texto.

#### Por que isso é importante?

- **Busca semântica (Semantic Search):** Ao comparar vetores de textos, é possível verificar quais são mais “semelhantes” no espaço vetorial. Assim, se tivermos vários documentos e quisermos encontrar aquele que mais se relaciona com uma consulta, basta comparar o embedding da consulta com o embedding de cada documento.
- **Clustering (agrupamento):** Podemos agrupar pedaços de texto que falem sobre assuntos semelhantes.
- **Classificação e análise de sentimento:** Embora existam métodos específicos para isso, embeddings também podem auxiliar em tarefas de classificação de textos, pois fornecem uma forma numérica de tratar as informações semânticas.

No contexto de processamento de linguagem natural (NLP), embeddings são a base para diversos sistemas de recomendação, análise de similaridade, chatbots e muito mais.

---

#### Como funcionam os Embeddings na LangChain?

A classe `Embeddings` da LangChain é uma classe-base para interfaces com vários provedores de modelos de embeddings, como OpenAI, Cohere e Hugging Face. Ela oferece uma padronização, fornecendo dois métodos principais:

1. **`.embed_documents`**
    
    - Recebe como entrada uma lista de textos (strings) e retorna uma **lista de embeddings** (cada embedding corresponde a um dos textos de entrada).
    - Exemplo de uso: criar embeddings para um conjunto de documentos em que se deseja fazer busca semântica.
2. **`.embed_query`**
    
    - Recebe um único texto (geralmente uma pergunta/consulta) e retorna um **vetor de floats** (uma única lista de números).
    - Exemplo de uso: criar um embedding para a query que será comparada com embeddings de documentos.

A diferenciação entre **documentos** e **query** existe porque, em alguns provedores de embeddings, há métodos distintos para processar cada tipo de texto. Além disso, separar as duas funções também deixa claro o fluxo: primeiro criamos embeddings dos documentos (que muitas vezes fica armazenado em algum index ou base de dados), depois criamos embeddings da consulta para comparar e ranquear os documentos mais relevantes.


#### Exemplo Prático com OpenAIEmbeddings 

Aqui, estamos usando o modelo `text-embedding-3-large` da OpenAI, mas existem outros modelos disponíveis (como `text-embedding-ada-002`, por exemplo). Cada modelo pode ter tamanhos diferentes de vetores, impactando a performance e a qualidade das representações.

#### Passo 4: Gerar Embeddings de Documentos

Suponha que você tenha uma lista de textos que deseja “indexar” ou armazenar para futuras buscas. Podemos usar `.embed_documents` para gerar os vetores correspondentes:

In [ ]:
%run ../helpers/00-llm.ipynb

In [ ]:
from helpers.llm import initialize_llm, logger, pretty_print
 
llm, _, embeddings = initialize_llm()

In [ ]:

documents = [
    "Olá!",
    "Quantos anos você tem?",
    "Qual seu nome?",
    "Meu amigo se chama flávio",
    "Oi!"
]

embeddings_docs = embeddings.embed_documents(documents)

print(embeddings_docs)
print(len(embeddings_docs[0]))



No exemplo da documentação, o retorno foi algo como `(5, 1536)`, indicando que temos 5 embeddings, cada um com 1536 valores.


#### Passo 5: Gerar Embedding de uma Query

Agora, se quisermos pesquisar algo relacionado a “qual é o nome mencionado na conversa?”, podemos criar um embedding específico para essa pergunta e depois comparar com os embeddings dos documentos:

In [ ]:
embedded_query = embeddings.embed_query("Qual é o nome do seu amigo?")
print(len(embedded_query))  # Tamanho do vetor da query (ex. 1536)
print(embedded_query[:5])

Esse vetor (`embedded_query`) pode então ser comparado com cada um dos vetores `embeddings` (dos documentos) usando uma métrica de similaridade, como cosseno (cosine similarity). Quanto maior a similaridade, mais relevante é o documento para a query.

#### O que Fazer Depois dos Embeddings?

1. **Armazenar em um Vetor (Vector Store):**  
    Existem diversas bibliotecas de indexação vetorial (ex.: FAISS, Milvus, Pinecone, Qdrant) que permitem armazenar e buscar embeddings de forma eficiente. Você insere cada embedding de documento na base, junto com um identificador do texto. Depois, quando recebe uma query, gera o embedding da query e faz a busca na base para recuperar os documentos mais similares.
    
2. **Buscar Documentos Relevantes:**  
    Após armazenar, podemos buscar documentos usando a similaridade de cosseno (ou outra métrica), retornando os mais próximos do embedding da query.
    
3. **Utilizar em um sistema RAG:**  
    Você pode, por exemplo, criar um chatbot que, com base nos documentos mais relevantes (encontrados por embeddings), gere respostas mais contextualizadas para o usuário.
    

#### Informação Extra:

##### 1. Diferença entre Modelos de Embedding

Modelos como `text-embedding-3-large` ou `text-embedding-ada-002` podem variar em termos de:

- **Dimensionalidade do vetor** (e.g., 768, 1536 dimensões).
- **Qualidade semântica**: alguns modelos capturam relações semânticas mais complexas do que outros.
- **Custo e tempo de inferência**: pode haver diferenças de preço por uso (no caso de APIs) ou tempo de computação em cada chamada.

##### 2. Melhores Práticas para Performance

- **Batching:** Quando for embedar múltiplos documentos, agrupe em lotes para reduzir chamadas sucessivas e otimizar.
- **Caching:** Se alguns documentos não mudam com frequência, armazene os embeddings em disco ou em um banco de dados para não precisar recalcular toda vez que iniciar a aplicação.

##### Conclusão

Os embeddings são peças fundamentais no universo de NLP quando precisamos lidar com similaridade textual, ranking de documentos, chatbots com contexto, entre outros cenários. A LangChain abstrai a lógica de criação de embeddings, permitindo que você troque de provedores (OpenAI, Cohere, Hugging Face) sem grandes refatorações no seu código.

Com um simples fluxo de:

1. **Gerar embeddings de documentos**
2. **Gerar embedding de query**
3. **Comparar** usando métricas de similaridade

… você já consegue criar sistemas de busca semântica e diversas soluções de linguagem natural de forma mais simples e estruturada.


#### Links auxiliares:

1 - Preço embeddings: https://openai.com/api/pricing/
2 - Modelos abertos no Huggingface: https://huggingface.co/models?other=text-embeddings-inference&sort=trending